<a href="https://colab.research.google.com/github/Muun-Muun/clusterd_sattelites_optimization/blob/main/impulse_ca_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from mpl_toolkits.mplot3d import Axes3D

# -------------------------------
# 1. 환경 설정
# -------------------------------
R_earth = 6378.0  # [km]
altitude = 300.0
r_mag = R_earth + altitude
mu = 398600.4418  # [km^3/s^2]
v_mag = np.sqrt(mu / r_mag)

# -------------------------------
# 2. ECI 초기 상태 설정
# -------------------------------
# Primary (경사각 0도, 적도 궤도)
r_primary = np.array([r_mag, 0, 0])
v_primary = np.array([0, v_mag, 0])
state_primary_0 = np.concatenate((r_primary, v_primary))

# Secondary (경사각 60도 궤도)
inclination_deg = 60
incl_rad = np.radians(inclination_deg)
r_secondary = r_primary  # 같은 반지름에서 출발
v_secondary = v_mag * np.array([0, np.cos(incl_rad), np.sin(incl_rad)])
state_secondary_0 = np.concatenate((r_secondary, v_secondary))

# -------------------------------
# 3. 적분 조건
# -------------------------------
T_orbit = 2 * np.pi * np.sqrt(r_mag**3 / mu)
t_vals = np.linspace(0, 0.5 * T_orbit, 1000)
t_span = (0, 0.5 * T_orbit)



# 적분 수행
sol_primary = solve_ivp(two_body_eci, t_span, state_primary_0, t_eval=t_vals, rtol=1e-9, atol=1e-9)
sol_secondary = solve_ivp(two_body_eci, t_span, state_secondary_0, t_eval=t_vals, rtol=1e-9, atol=1e-9)

traj_prim = sol_primary.y.T
traj_secd = sol_secondary.y.T


In [ ]:
def project_to_bplane(state_primary, state_secondary):
    # 1. 상태벡터 분리
    r1, v1 = state_primary[:3], state_primary[3:]
    r2, v2 = state_secondary[:3], state_secondary[3:]

    # 2. 상대 위치 및 속도
    r_rel = r2 - r1
    v_rel = v2 - v1

    # 3. b-plane 좌표계 정의
    z_b = v_rel / np.linalg.norm(v_rel)
    h = np.cross(r1, v1)
    y_b = np.cross(z_b, h)
    y_b = y_b / np.linalg.norm(y_b)
    x_b = np.cross(y_b, z_b)

    # 4. 회전 행렬 (b-plane to ECI)
    R_b2eci = np.vstack([x_b, y_b, z_b]).T
    R_eci2b = R_b2eci.T

    # 5. 투영 (ECI → b-plane)
    r_b = R_eci2b @ r_rel
    v_b = R_eci2b @ v_rel

    state_bplane_3d = np.concatenate((r_b, v_b))
    state_bplane_2d = np.concatenate((r_b[:2], v_b[:2]))  # b-plane 평면 성분만

    return state_bplane_3d, state_bplane_2d


  def two_body_eci(t, state):
    r = state[:3]
    v = state[3:]
    norm_r = np.linalg.norm(r)
    a = -mu * r / norm_r**3
    return np.concatenate((v, a))

In [19]:
import plotly.graph_objects as go


# 지구 시각화용 구
u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:25j]
x_sphere = R_earth * np.cos(u) * np.sin(v)
y_sphere = R_earth * np.sin(u) * np.sin(v)
z_sphere = R_earth * np.cos(v)


# Primary 궤도 (0°)
trace_primary = go.Scatter3d(
    x=traj_prim[:, 0], y=traj_prim[:, 1], z=traj_prim[:, 2],
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Inclination 0°'
)

# Secondary 궤도 (60°)
trace_secondary = go.Scatter3d(
    x=traj_secd[:, 0], y=traj_secd[:, 1], z=traj_secd[:, 2],
    mode='lines',
    line=dict(color='blue'),
    name='Inclination 60°'
)

# 시작점
start_primary = go.Scatter3d(
    x=[traj_prim[0, 0]], y=[traj_prim[0, 1]], z=[traj_prim[0, 2]],
    mode='markers', marker=dict(color='red', size=5, symbol='x'),
    name='Start 0°'
)

start_secondary = go.Scatter3d(
    x=[traj_secd[0, 0]], y=[traj_secd[0, 1]], z=[traj_secd[0, 2]],
    mode='markers', marker=dict(color='blue', size=5, symbol='x'),
    name='Start 60°'
)

# 종말점
start_primary = go.Scatter3d(
    x=[traj_prim[-1, 0]], y=[traj_prim[-1, 1]], z=[traj_prim[0, 2]],
    mode='markers', marker=dict(color='red', size=5, symbol='circle'),
    name='Start 0°'
)

start_secondary = go.Scatter3d(
    x=[traj_secd[-1, 0]], y=[traj_secd[-1, 1]], z=[traj_secd[0, 2]],
    mode='markers', marker=dict(color='blue', size=5, symbol='circle'),
    name='Start 60°'
)

# 지구 구
sphere = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    colorscale=[[0, 'lightblue'], [1, 'lightblue']],
    opacity=0.3,
    showscale=False
)

layout = go.Layout(
    title='Comparison of 0° and 60° Inclined Orbits (3 Periods)',
    scene=dict(
        xaxis_title='ECI X [km]',
        yaxis_title='ECI Y [km]',
        zaxis_title='ECI Z [km]',
        aspectmode='data'
    )
)

fig = go.Figure(data=[sphere, trace_primary, trace_secondary, start_primary, start_secondary], layout=layout)
fig.show()


In [27]:


project_to_bplane(traj_prim[-1], traj_secd[-1])



(array([ 5.03504047e-06, -3.78001434e-07,  3.37251964e-05, -1.75800233e-16,
         0.00000000e+00,  7.72583948e+00]),
 array([ 5.03504047e-06, -3.78001434e-07, -1.75800233e-16,  0.00000000e+00]))